# Examine a single word  


In [ ]:
import os
import troubleshooting 
import data_wrangling
import tensorflow as tf
from IPython.display import clear_output
from ipywidgets import interact
import matplotlib.pyplot as plt


# Intactive input/act

# Examine one

In [ ]:
batch_name = "task_effect"
code_name = 'task_effect_r0017'
testset_name = 'train_r100'
d = troubleshooting.Diagnosis(f"{batch_name}/{code_name}", tf_root_override='/home/jupyter/triangle_model')
d.eval(testset_name, task='triangle', epoch=1000)


In [ ]:

@interact(
    sel_word=d.testset_package['item'], 
    layer=['pho', 'sem'], 
    task=['triangle', 'ort_pho', 'exp_osp', 'ort_sem', 'exp_ops'], 
    epoch=(d.cfg.save_freq, d.cfg.total_number_of_epoch + 1, d.cfg.save_freq)
    )
def interactive_plot(sel_word, layer, task, epoch):
    d = troubleshooting.Diagnosis(f"{batch_name}/{code_name}", tf_root_override='/home/jupyter/triangle_model')
    d.eval(testset_name, task=task, epoch=epoch)
    d.set_target_word(sel_word)
    print(f"Output phoneme over timeticks: {d.list_output_phoneme}")
    return d.plot_one_layer(layer)

## Explore Cosine vs. target
Run cosine(sigmoid(input), target) 

In [ ]:
class MyCosine:
    
    def __init__(self, y_true):
        self.y_true = y_true

    @staticmethod
    def sigmoid(x):
        return tf.keras.activations.sigmoid(x)

    def cosine(self, y_pred):
        return -tf.keras.losses.cosine_similarity(self.y_true, y_pred)

    def cosine_on_time_axis(self, ys):
        return tf.map_fn(self.cosine, ys)

    def __call__(self, ys, apply_sigmoid=False):

        if apply_sigmoid:
            ys = self.sigmoid(ys)

        return tf.reduce_mean(self.cosine_on_time_axis(ys), axis=-1).numpy()

    def plot(self, ys, apply_sigmoid=False):
        plt.plot(self.__call__(ys, apply_sigmoid))
        plt.show()

# Instantiating the cosine metric
cos = MyCosine(y_true=d.testset_package['sem'])

### SEM

In [ ]:
ps = cos(d.y_pred['input_hps_hs'], apply_sigmoid=True)
cs = cos(d.y_pred['input_css_cs'], apply_sigmoid=True)
os = cos(d.y_pred['input_hos_hs'], apply_sigmoid=True)
inp = cos(d.y_pred['input_sem'], apply_sigmoid=True)
act_s = cos(d.y_pred['sem'], apply_sigmoid=False)

plt.plot(ps, label='ps')
plt.plot(cs, label='cs')
plt.plot(os, label='os')
plt.plot(inp, label='sum_input')
plt.plot(act_s, label='s')
plt.legend()

### PHO

In [ ]:
cos = MyCosine(y_true=d.testset_package['pho'])

sp = cos(d.y_pred['input_hsp_hp'], apply_sigmoid=True)
cp = cos(d.y_pred['input_cpp_cp'], apply_sigmoid=True)
op = cos(d.y_pred['input_hop_hp'], apply_sigmoid=True)
inp = cos(d.y_pred['input_pho'], apply_sigmoid=True)
act_p = cos(d.y_pred['pho'], apply_sigmoid=False)

plt.plot(sp, label='sp')
plt.plot(cp, label='cp')
plt.plot(op, label='op')
plt.plot(inp, label='sum_input')
plt.plot(act_p, label='p')
plt.legend()

In [ ]:
-tf.keras.losses.cosine_similarity([1., 1.], [0.1, 0.1])

In [ ]:
SEM_NAME_MAP = {
    "input_hps_hs": "PS",
    "input_css_cs": "CS",
    "input_hos_hs": "OS",
    "input_sem": "input",
    "sem": "act",
}

PHO_NAME_MAP = {
    "input_hsp_hp": "SP",
    "input_cpp_cp": "CP",
    "input_hop_hp": "OP",
    "input_pho": "input",
    "pho": "act",
}

## Hand crafted version for better labeling 

In [ ]:
ps = d.y_pred['input_hps_hs']
cs = d.y_pred['input_css_cs']
os = d.y_pred['input_hos_hs']
s = d.y_pred['sem']
sum_input = d.y_pred['input_sem']
y_true = d.testset_package['sem']

# Pho

## Scales:
- 1: Identical
- 0: 90 degrees
- -1: 180 degree